# **Notes**
### Analogue analysis for HadGEM3 model streamfunction data
#### Also makes the violin and intensity plots

* (Folder should only have files containing data from 1950-2014?)
* The AMO functions are not propely tested. When using anything other than the violin plots and intensity plots (the plots that made it into the actual report), I would recommend to double-check the code.

# **Preparation**






In [1]:
import numpy as np # for storing vector and matrix data
import matplotlib.pyplot as plt # to plot figures
import netCDF4 as nc #to read netCDF files
import cartopy.crs as ccrs # to plot maps
# (ergens in test ook: import cartopy as cart)
import cartopy.feature as cf
# from matplotlib import ticker
import scipy.io
from scipy.stats import pearsonr # voor persistence
import scipy.stats as stats
# from cartopy.util import add_cyclic_point
import os
from matplotlib.gridspec import GridSpec


# **Functions**

In [2]:
def load_data(path,*variables_to_add):
  """
Provide the path to a file and the variables you want to extract
  """
  data = nc.Dataset(path, mode='r')
  variable_list = []
  for variable in variables_to_add:
    var =data.variables[variable][:]
    variable_list.append(var)
  return variable_list

In [3]:
def extract_area(S, N, W, E, lat, lon, variable,event = False):
    """
    This function slices the data given the S, N, W, E bounds. Use event = True if there are only two dimensions (since then there is no time dimension), this means after using this
    function you need to use event[0] to get the data
    """
    # Change longitude data to go from -180 to 180
    for i in range(len(lon)):
        if lon[i] > 180:
          lon[i] = lon[i] - 360
        else:
          lon[i] = lon[i]

    # Calculate the index of the bounds
    sIndex = np.argmin(np.abs(lat - S))
    nIndex = np.argmin(np.abs(lat - N))
    wIndex = np.argmin(np.abs(lon - W))
    eIndex = np.argmin(np.abs(lon - E))

    if event:
        variable = np.expand_dims(variable, axis = 0)

    if wIndex > eIndex: # If the west index is higher than the east index, think of the right side of the world map as left boundary and vice versa
        latSlice = lat[sIndex: nIndex + 1]
        lonSlice = np.concatenate((lon[wIndex:], lon[:eIndex + 1]))
        variableSlice = np.concatenate((variable[:, sIndex: nIndex + 1, wIndex:], variable[:, sIndex: nIndex + 1, :eIndex + 1]), axis = 2)

    else:
        latSlice = lat[sIndex: nIndex + 1]
        lonSlice = lon[wIndex: eIndex + 1]
        variableSlice = variable[:, sIndex: nIndex + 1, wIndex: eIndex + 1]

    return latSlice, lonSlice, variableSlice

In [4]:
def extract_season_year(variable,yearlength,start_day,end_day, start_year = False, end_year = False):
  """
  Start day and end day should be the actuall day, so if you want the second day, third and fourth day, do 2,4 (151,240 would be JJA?) jaren ook: 1,5 is 1 tot en met 5
  nadenken dat als gaat checken met al gesneden data dat yearlenght 90 is als op 3 maanden gesneden
  """
  start_index = start_day-1
  end_index = end_day-1
  if start_year == False and end_year == False:
    years = variable.shape[0]//yearlength
    for year in range(years):
      if year == 0:
        selected_data = variable[(year*360)+start_index:(end_index+1),:,:] # +1 omdat tot is ipv tot en met voor de laatste
      elif year != 0:
        add_data = variable[(year*360)+start_index:(year*360)+(end_index+1),:,:] # stel is 10, na 1 jaar dan 370 is TOT 370 dus index 369 en dan is dag 10
        selected_data = np.concatenate((selected_data, add_data), axis = 0)
    return selected_data
  else:
    years = (end_year-start_year) + 1
    for year in range(years):
      year_multiplier = (year + start_year) - 1
      if year == 0:
        selected_data = variable[(year_multiplier*360)+start_index:(year_multiplier*360)+(end_index+1),:,:] # +1 omdat tot is ipv tot en met voor de laatste
      elif year != 0:
        add_data = variable[(year_multiplier*360)+start_index:(year_multiplier*360)+(end_index+1),:,:] # stel is 10, na 1 jaar dan 370 is TOT 370 dus index 369 en dan is dag 10
        selected_data = np.concatenate((selected_data, add_data), axis = 0)
    return selected_data

In [5]:
def euclidian_distance(data, event):
  """
Calculates the euclidian distance for each day in the data compared to a given single event, gives an array (~list) of all the distances in chronological order
  """
  return np.sqrt(np.sum((data - event)**2, axis = (1, 2)))

In [6]:
def determine_analogues(euclidian_distances, target_number_of_analogues,analogue_seperation_range):
  """
  Determine the best analogues. Give an array of all unsorted euclidan distances. The analogue_seperation_range determines how many days have to seperate
  the different analogues (if analogue_seperation_range = 5, the fifth day after an analogue can also still not be a new analogue(dus die dag mag ook niet)).
  Target_number_of_analogues is how many analogues are selected. Returns the indexes of the best analogues in the original data, and the euclidian distances corresponding to the analogues.
  """
  distance_index_dictionary = {value: index for index, value in enumerate(euclidian_distances)} # Gives the index in the original euclidian distances list for a value in the sorted list
  sorted_distances = np.sort(euclidian_distances) # sort the distances from low to high euclidian distances
  analogues_index_list = [] # create a list to save the indexes of the selected analogues
  euclidian_distance_list = [] # create a list to save the euclidian distances of the selected analogues
  i=0 #index counter
  while len(analogues_index_list) < target_number_of_analogues and (i < len(euclidian_distances)):
    differences= []
    index = distance_index_dictionary[sorted_distances[i]]
    if len(analogues_index_list) == 0:
      analogues_index_list.append(index)
      euclidian_distance_list.append(sorted_distances[i])
      i = i + 1
    else:
      for item in analogues_index_list:
        difference = (index-item)
        if difference < (-1*analogue_seperation_range) or difference > analogue_seperation_range:
          differences.append(2) #goed
        elif difference >= (-1*analogue_seperation_range) and difference <= (analogue_seperation_range):
          differences.append(1) #niet goed
      if min(differences) == 2:
        analogues_index_list.append(index)
        euclidian_distance_list.append(sorted_distances[i])
        i = i + 1
      elif min(differences) == 1:
        i = i + 1

  return analogues_index_list,euclidian_distance_list

In [7]:
def persistence(analogue_indexes,data,c_threshold):
  """
Defines the persistence of each analogue based on a correlation threshold (>=). It returns a list with a list for each analogue, -1 is a day before the event, 1 is a day after the event.
So if you want the total lenght of the event you still need to add the event itself? (nu wel in lenghts gedaan :) Also returns the correlations in the correct order.
(als je dus de analogues bepaald heb met data gesliced op seizoen die ook hier gebruiken?)
  """
  list_for_lists = [] #
  list_for_lists_values = []
  # nu 500 ook laatste dag dus kan niet vooruit
  for index in analogue_indexes:
    list_for_length = []
    list_for_values = []
    analogue_map = data[index]
    Ab,Ac = np.shape(analogue_map)
    analogue_af = analogue_map.reshape(Ab*Ac)

    index_dag1 = index + 1
    index_dag2 = index - 1
    correlation = 1

    while correlation >= c_threshold and index_dag2 >= 0:
      day_data = data[index_dag2]
      Bb,Bc = np.shape(day_data)
      day_map_af = day_data.reshape(Bb*Bc)
      correlation = pearsonr(analogue_af, day_map_af)[0]
      if correlation >= c_threshold:
        list_for_length.insert(0,-1)
        list_for_values.insert(0,correlation) # dit is met insert dat is anders, dan doet ie het aan de voorkant dus dan staan de values echt op volgorde van de dag.
      index_dag2 = index_dag2 - 1

    correlation = 1

    while correlation >= c_threshold and index_dag1 <= (len(data)-1):
      day_data = data[index_dag1]
      Bb,Bc = np.shape(day_data)
      day_map_af = day_data.reshape(Bb*Bc)
      correlation = pearsonr(analogue_af, day_map_af)[0]
      if correlation >= c_threshold:
        list_for_length.append(1)
        list_for_values.append(correlation)
      index_dag1 = index_dag1 + 1


    list_for_lists.append(list_for_length) # nu dus niet anaologue zelf meegenomen in lengte dus voor lengte nog lengte lijst +1
    list_for_lists_values.append(list_for_values)

  list_for_lengths = []
  for lijst in list_for_lists:
    lengte = len(lijst) + 1
    list_for_lengths.append(lengte)

  return list_for_lists, list_for_lists_values,list_for_lengths

In [8]:
def plot_variable(lat, lon, variable,folder,name):
    plt.figure(figsize = (10,10))
    ax = plt.axes(projection = ccrs.PlateCarree())
    plot = plt.contourf(lon, lat, variable, cmap = "RdBu_r", transform = ccrs.PlateCarree(), levels = 15) #levels=np.linspace(-8.2e7, 1e7, 10), extend='both
    ax.coastlines()
    ax.add_feature(cf.BORDERS)
    plt.colorbar(plot, ax=ax, orientation = "horizontal", label = "Pressure (Pa)", pad = 0.05)
    plt.savefig(f"{folder}/{name}.png",dpi=300)
    #plt.show()
    plt.close()

In [9]:
def determine_analogues_era5(euclidian_distances, target_number_of_analogues,analogue_seperation_range,list_months,list_years,desired_start_month,desired_end_month,event_is_in_data = True):
  """
  Determine the best analogues. Give an array of all unsorted euclidan distances. The analogue_seperation_range determines how many days have to seperate
  the different analogues (if analogue_seperation_range = 5, the fifth day after an analogue can also still not be a new analogue(dus die dag mag ook niet)).
  Target_number_of_analogues is how many analogues are selected. Returns the indexes of the best analogues in the original data, and the euclidian distances corresponding to the analogues.
  Give semi-filtered data to euclidian_distances: filtered on years but all months, so the seperation range doesn't continue to count in the next season
  Should give semi-filtered lists of months and years as well??
  """
  distance_index_dictionary = {value: index for index, value in enumerate(euclidian_distances)} # Gives the index in the original euclidian distances list for a value in the sorted list
  sorted_distances = np.sort(euclidian_distances) # sort the distances from low to high euclidian distances
  analogues_index_list = [] # create a list to save the indexes of the selected analogues
  euclidian_distance_list = [] # create a list to save the euclidian distances of the selected analogues
  selected_analogue_years = []
  if event_is_in_data == False:
    i = 0
    if sorted_distances[0] == 0:
      print ("WARNING: event does seem to be in data, while event_is_in_data == False (zelf)")
    while len(analogues_index_list) < target_number_of_analogues and (i < len(euclidian_distances)):
      differences= []
      index = distance_index_dictionary[sorted_distances[i]]
      month = list_months[index]
      year = list_years[index]
      if len(analogues_index_list) == 0:
        if month >= desired_start_month and month <= desired_end_month:
          analogues_index_list.append(index)
          euclidian_distance_list.append(sorted_distances[i])
          selected_analogue_years.append(year)
        i = i + 1
      else:
        if month >= desired_start_month and month <= desired_end_month:
          for item in analogues_index_list:
            difference = (index-item)
            if difference < (-1*analogue_seperation_range) or difference > analogue_seperation_range:
              differences.append(2) #goed
            elif difference >= (-1*analogue_seperation_range) and difference <= (analogue_seperation_range):
              differences.append(1) #niet goed
          if min(differences) == 2:
            analogues_index_list.append(index)
            euclidian_distance_list.append(sorted_distances[i])
            selected_analogue_years.append(year)
            i = i + 1
          elif min(differences) == 1:
            i = i + 1
        else:
          i = i + 1

  elif event_is_in_data == True: #Need to make sure the selected analogues are also 5 days away from the event, eventhough it is not a selecgted analogue 
    event_index = distance_index_dictionary[sorted_distances[0]]
    analogues_index_list.append(event_index)
    i=1 #index counter
    if sorted_distances[0] != 0:
      print ("WARNING: event does not seem to be in data, while event_is_in_data == True (zelf)")
    while len(analogues_index_list) < (target_number_of_analogues+1) and (i < len(euclidian_distances)): #+1 omdat event er nu ook in en die later weghalen
      differences= []
      index = distance_index_dictionary[sorted_distances[i]]
      month = list_months[index]
      year = list_years[index]
      if len(analogues_index_list) == 0:
        if month >= desired_start_month and month <= desired_end_month:
          analogues_index_list.append(index)
          euclidian_distance_list.append(sorted_distances[i])
          selected_analogue_years.append(year)
        i = i + 1
      else:
        if month >= desired_start_month and month <= desired_end_month:
          for item in analogues_index_list:
            difference = (index-item)
            if difference < (-1*analogue_seperation_range) or difference > analogue_seperation_range:
              differences.append(2) #goed
            elif difference >= (-1*analogue_seperation_range) and difference <= (analogue_seperation_range):
              differences.append(1) #niet goed
          if min(differences) == 2:
            analogues_index_list.append(index)
            euclidian_distance_list.append(sorted_distances[i])
            selected_analogue_years.append(year)
            i = i + 1
          elif min(differences) == 1:
            i = i + 1
        else:
          i = i + 1
    analogues_index_list.pop(0)

  if len(analogues_index_list) < target_number_of_analogues:
    print ("WARNING: not enough data to find the required amount of analogues (zelf)")
  if len(analogues_index_list) != target_number_of_analogues:
    print ("WARNING: not the right amount of analogues has been found")
  return analogues_index_list,euclidian_distance_list,selected_analogue_years

In [10]:
def typicality(analogue_distances, analogue_indexes,data,target_number_of_analogues,analogue_seperation_range,month_list,year_list,season_start,season_end):
  """
Calculates the Tevent en Tanalogue. Distances need to be from the selected analogues only. Data has to be the one used to calculate the original analogues.
  """
  Tevent = 1/(sum(analogue_distances))
  list_for_Tanalogue = []
  for i in range(len(analogue_indexes)):
    index_in_used_data = analogue_indexes[i]
    euclidian_distance_for_analogue = euclidian_distance(data,data[index_in_used_data])
    index_for_analogues_for_analogue,distance_for_analogues_for_analogue, analogues_years = determine_analogues_era5(euclidian_distance_for_analogue, target_number_of_analogues,analogue_seperation_range,month_list,year_list,season_start,season_end,event_is_in_data=True)
    Tanalogue = 1/(sum(distance_for_analogues_for_analogue))
    list_for_Tanalogue.append(Tanalogue)
  return Tevent, list_for_Tanalogue

In [11]:
def violin_plot(Tevent_past,Tevent_present,Tanalogue_past,Tanalogue_present,persistence_past,persistence_present,folder,name):
  """
Doet het? maar stipjes van persistence standaard op 1 want hebt nog niet van event?
  """
  u, p = stats.ttest_ind(Tanalogue_past, Tanalogue_present) #calculate p value
  text_to_plot = f"p = {p:.3f}"

  fig, (ax1,ax2) = plt.subplots(1,2,figsize = (8, 5))
  violins = ax1.violinplot([Tanalogue_past, Tanalogue_present], [1, 1.6], showmeans=False, showextrema=False, showmedians=False)
  colors = ["magenta", "green"]
  for pc, color in zip(violins["bodies"], colors): #pc zijn de violins dus moet zo als verschillende kleuren wil
    pc.set_facecolor(color)
  ax1.plot(1,Tevent_past, marker = "o", color = "r") #plot de events
  ax1.plot(1.6,Tevent_present, marker = "o", color = "r")  #plot de events
  ax1.axhline(np.mean(Tanalogue_past), color = colors[0], linewidth = 3)
  ax1.axhline(np.mean(Tanalogue_present), color = colors[1], linewidth = 3)
  ax1.set_xticks([1, 1.6])
  ax1.set_xticklabels(["Past", "Present"])
  ax1.set_ylim(top = (1.035*(max(max(Tanalogue_past),max(Tanalogue_present))))) # to ensure that the plotted p-value and graph do not overlap
  ax1.text(0.035,0.95,text_to_plot,transform=ax1.transAxes)

  u, p = stats.ttest_ind(persistence_past, persistence_present) #calculate p value
  text_to_plot = f"p = {p:.3f}"

  violins = ax2.violinplot([persistence_past, persistence_present], [1, 1.6], showmeans=False, showextrema=False, showmedians=False)
  colors = ["magenta", "green"]
  for pc, color in zip(violins["bodies"], colors): #pc zijn de violins dus moet zo als verschillende kleuren wil
    pc.set_facecolor(color)
  ax2.plot(1,1, marker = "o", color = "r") #plot de events
  ax2.plot(1.6,1, marker = "o", color = "r")  #plot de events
  ax2.axhline(np.mean(persistence_past), color = colors[0], linewidth = 3)
  ax2.axhline(np.mean(persistence_present), color = colors[1], linewidth = 3)
  ax2.set_xticks([1, 1.6])
  ax2.set_xticklabels(["Past", "Present"])
  ax2.set_ylim(top = (1.07*(max(max(persistence_past),max(persistence_present))))) # to ensure that the plotted p-value and graph do not overlap
  ax2.text(0.035,0.95,text_to_plot,transform=ax2.transAxes)

  plt.savefig(f"{folder}/{name}.png",dpi=300)
  plt.close()
  #plt.show()

In [12]:
def yearly_minimum_euclidian_distance(euclidian_distances, yearlength = 360):
  """
Finds the best analogue (lowest ecleudian distance) for each year to do trend analysis. Returns an array with the distances.
  """
  distances_per_year = euclidian_distances.reshape((euclidian_distances.shape[0] // yearlength, yearlength))
  return np.min(distances_per_year, axis = 1) #loopt blijkbaar al automatisch door alle jaren

In [13]:
def plot_trend(yearly_min_distance,distances, running_mean_window, start_year, end_year, folder, name):
  """
Moet wel goed checken dat length x_ticks and y_data hetzelfde is (dus 100 years, is 1850 tot en met 1949! dus ook end_year = 1949)
  """
  plt.figure(figsize = (10,8))
  max_distance = max(distances)
  y_data = 1 - (yearly_min_distance/max_distance) # loopt ook somehow al door alle distances
  running_mean = np.convolve(y_data, np.ones(running_mean_window) / running_mean_window, mode = "valid") #valid is dat rand niet doet (kan wel met 'same' maar dan voegt die nullen toe dus dan average fout,). Kernel (2e input) is weight voor verschillende plekken in window (hier overal gelijk?)
  x_ticks = range(start_year, end_year + 1)
  plt.ylabel("1 - (ED/EDmax)")
  plt.xlabel("Years")

  plt.plot(x_ticks,y_data)
  plt.plot(range(start_year + (running_mean_window // 2 - 1), end_year - (running_mean_window // 2 - 1)),running_mean) #begin van bram, omdat eerste paar jaar niet window kan doen
  plt.savefig(f"{folder}/{name}.png",dpi=300)
  #plt.show()
  plt.close()

  return y_data, running_mean

In [14]:
def create_directories(base_path,name_run,extra_folder1 = "figures",extra_folder2 = "lists",figure_folders = []):
  """
Create a new folder to save the outputs
  """
  new_directory_path = os.path.join(base_path, name_run)
  if not os.path.exists(new_directory_path):
    os.makedirs(new_directory_path)

  figure_folder_path = os.path.join(new_directory_path, extra_folder1)
  if not os.path.exists(figure_folder_path):
    os.makedirs(figure_folder_path)

  list_folder_path = os.path.join(new_directory_path, extra_folder2)
  if not os.path.exists(list_folder_path):
    os.makedirs(list_folder_path)
  
  violin_plot_path = os.path.join(figure_folder_path, "violin_plots")
  if not os.path.exists(violin_plot_path):
    os.makedirs(violin_plot_path)

  analogue_plot_path = os.path.join(figure_folder_path, "analogue_plots")
  if not os.path.exists(analogue_plot_path):
    os.makedirs(analogue_plot_path)

  difference_plot_path = os.path.join(figure_folder_path, "difference_plots")
  if not os.path.exists(difference_plot_path):
    os.makedirs(difference_plot_path)

  trend_plot_path = os.path.join(figure_folder_path, "trend_plots")
  if not os.path.exists(trend_plot_path):
    os.makedirs(trend_plot_path)

  AMO_plot_path = os.path.join(figure_folder_path, "AMO_plots")
  if not os.path.exists(AMO_plot_path):
    os.makedirs(AMO_plot_path)

  return new_directory_path,figure_folder_path,list_folder_path,violin_plot_path,analogue_plot_path,difference_plot_path,trend_plot_path,AMO_plot_path

In [15]:
def mean_difference_per_ensemble(analogue_indexes_1850,data_1850,analogue_indexes_1950,data_1950):
  list_for_1850 = []
  list_for_1950 = []

  for index in analogue_indexes_1850:
    data_analogue = data_1850[index]
    list_for_1850.append(data_analogue)

  for index in analogue_indexes_1950:
    data_analogue = data_1950[index]
    list_for_1950.append(data_analogue)

  array_1850 = np.array(list_for_1850) #maak van de list weer een array
  mean_array_1850 = np.mean(array_1850, axis = 0)

  array_1950 = np.array(list_for_1950) #maak van de list weer een array
  mean_array_1950 = np.mean(array_1950, axis = 0)

  difference_per_ensemble = mean_array_1950 - mean_array_1850
  return difference_per_ensemble, mean_array_1850, mean_array_1950, list_for_1850, list_for_1950

In [16]:
def combined_difference_top_analogues(all_analogue_fields,all_analogue_distances,target_number_of_analogues):
  """
all_analogue_fields should be a list with lists, with each inner list containing the fields of all selected analogues for one ensemble member. all_analogue_distances should be the same but with the euclidian distances of the selected analogues.
Returns the mean of the top combined selected analogues from all ensemble members combined (so for example the best 30 analogues when taking into account all ensemble members at once) for the difference plot.
Also returns the list with all the best analogue fields, for the t-test.
  """
  best_combined_analogue_fields = []
  best_combined_distances_list = []

  complete_list_fields = []
  for analogue_list in all_analogue_fields:
    for field in analogue_list:
      complete_list_fields.append(field)

  complete_list_distances = []
  for distance_list in all_analogue_distances:
    for distance in distance_list:
      complete_list_distances.append(distance)

  distance_index_dictionary = {value: index for index, value in enumerate(complete_list_distances)} # Gives the index in the original euclidian distances list for a value in the sorted list
  sorted_distances = sorted(complete_list_distances)
  for i in range(target_number_of_analogues):
    index = distance_index_dictionary[sorted_distances[i]]
    data_to_add = complete_list_fields[index]
    best_combined_analogue_fields.append(data_to_add)
    distance_to_add = sorted_distances[i]
    best_combined_distances_list.append(distance_to_add)

  array_best_combined_fields = np.array(best_combined_analogue_fields) #maak van de list weer een array
  ensembles_combined_mean_array = np.mean(array_best_combined_fields, axis = 0)

  return ensembles_combined_mean_array,best_combined_analogue_fields,best_combined_distances_list

In [17]:
def t_test(data1850,data1950,p_value):
  """
Performs a two sides t-test, for a analogue difference map data1850 and data1950 should be lists with arrays for each analogue (so not already the mean!), p_value should be for example 0.05
  """
  number_of_analogues = len(data1850)
  significance_mask = data1850[0].copy()
  a, b = np.shape(data1850[0])
  for i in range(a): # for a x
    #print(i)
    for j in range(b): # check every y on that x
      lijst1850 = []
      lijst1950 = []
      for R in range(number_of_analogues): # for each combination of x and y (so each cell) check each analogue
        lijst1850.append(data1850[R][i,j])
        lijst1950.append(data1950[R][i,j])
      #u, p = stats.mannwhitneyu(loc_list1,loc_list2)
      u, p = stats.ttest_ind(lijst1850, lijst1950)
      if p < p_value:
        significance_mask[i,j] = 1
      else:
        significance_mask[i,j] = 0
  return significance_mask

In [18]:
def plot_difference(lat, lon, variable,significance_mask,folder,name):
    plt.figure(figsize = (10,10))
    ax = plt.axes(projection = ccrs.PlateCarree())
    plot = plt.contourf(lon, lat, variable, cmap = "RdBu_r", transform = ccrs.PlateCarree(), levels = 15) #levels=np.linspace(-8.2e7, 1e7, 10), extend='both
    ax.contourf( lon, lat,significance_mask, levels=[-2,0,2], hatches=[None, '////'], colors='none', transform=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cf.BORDERS)
    plt.colorbar(plot, ax=ax, orientation = "horizontal", label = "Pressure (Pa)", pad = 0.05)
    plt.savefig(f"{folder}/{name}.png",dpi=300)
    #plt.show()
    plt.close()

In [19]:
def typicality_combined(analogue_distances,analogue_fields,data,target_number_of_analogues,analogue_seperation_range):
  """
Calculates the Tevent en Tanalogue for the top analogues
  """
  Tevent = 1/(sum(analogue_distances))
  list_for_Tanalogue = []
  for i in range(len(analogue_fields)):
    analogue_to_set_as_event = analogue_fields[i]
    euclidian_distance_for_analogue = euclidian_distance(data,analogue_to_set_as_event)
    index_for_analogues_for_analogue,distance_for_analogues_for_analogue = determine_analogues(euclidian_distance_for_analogue, target_number_of_analogues,analogue_seperation_range)
    Tanalogue = 1/(sum(distance_for_analogues_for_analogue))
    list_for_Tanalogue.append(Tanalogue)
  return Tevent, list_for_Tanalogue

In [20]:
def typicality_combined_new(analogue_distances, analogue_fields,data,target_number_of_analogues,analogue_seperation_range,month_list,year_list,season_start,season_end):
  """
Calculates the Tevent en Tanalogue. Distances need to be from the selected analogues only. Data has to be the one used to calculate the original analogues.
  """
  Tevent = 1/(sum(analogue_distances))
  list_for_Tanalogue = []
  for i in range(len(analogue_fields)):
    analogue_to_set_as_event = analogue_fields[i]
    euclidian_distance_for_analogue = euclidian_distance(data,analogue_to_set_as_event)
    index_for_analogues_for_analogue,distance_for_analogues_for_analogue, analogues_years = determine_analogues_era5(euclidian_distance_for_analogue, target_number_of_analogues,analogue_seperation_range,month_list,year_list,season_start,season_end,event_is_in_data=True)
    Tanalogue = 1/(sum(distance_for_analogues_for_analogue))
    list_for_Tanalogue.append(Tanalogue)
  return Tevent, list_for_Tanalogue

In [21]:
def persistence_combined(analogue_fields,data,c_threshold,yearlength_filtered):
  """
Defines the persistence of each analogue based on a correlation threshold (>=). It returns a list with a list for each analogue, -1 is a day before the event, 1 is a day after the event.
So if you want the total lenght of the event you still need to add the event itself? (nu wel in lenghts gedaan :) Also returns the correlations in the correct order.
(als je dus de analogues bepaald heb met data gesliced op seizoen die ook hier gebruiken?)
  """

  list_for_lists = [] #
  list_for_lists_values = []
  analogue_indexes = []

  for analogue_field in analogue_fields:
    index_counter = 0
    match_condition = False
    while match_condition == False and index_counter < (data.shape[0]):
      data_field = data[index_counter]
      if np.array_equal(data_field, analogue_field) == True:
        analogue_indexes.append(index_counter)
        match_condition = True
      else:
        index_counter = index_counter + 1

  # nu 500 ook laatste dag dus kan niet vooruit
  for index in analogue_indexes:
    list_for_length = []
    list_for_values = []
    analogue_map = data[index]
    Ab,Ac = np.shape(analogue_map)
    analogue_af = analogue_map.reshape(Ab*Ac)

    index_dag1 = index + 1
    index_dag2 = index - 1
    correlation = 1
    amount_of_years_passed = index//yearlength_filtered # To ensure that an analogue cant continue to persist in data from a different ensemble member
    max_index_threshold_forward = ((amount_of_years_passed+1)*yearlength_filtered) # moet kleiner zijn dan dat (want index 0-89 is 1 jaar)
    min_index_threshold_forward = amount_of_years_passed*yearlength_filtered #die mag nog wel
    while correlation >= c_threshold and index_dag2 >= min_index_threshold_forward:
      day_data = data[index_dag2]
      Bb,Bc = np.shape(day_data)
      day_map_af = day_data.reshape(Bb*Bc)
      correlation = pearsonr(analogue_af, day_map_af)[0]
      if correlation >= c_threshold:
        list_for_length.insert(0,-1)
        list_for_values.insert(0,correlation) # dit is met insert dat is anders, dan doet ie het aan de voorkant dus dan staan de values echt op volgorde van de dag.
      index_dag2 = index_dag2 - 1

    correlation = 1

    while correlation >= c_threshold and index_dag1 < max_index_threshold_forward:
      day_data = data[index_dag1]
      Bb,Bc = np.shape(day_data)
      day_map_af = day_data.reshape(Bb*Bc)
      correlation = pearsonr(analogue_af, day_map_af)[0]
      if correlation >= c_threshold:
        list_for_length.append(1)
        list_for_values.append(correlation)
      index_dag1 = index_dag1 + 1


    list_for_lists.append(list_for_length) # nu dus niet anaologue zelf meegenomen in lengte dus voor lengte nog lengte lijst +1
    list_for_lists_values.append(list_for_values)

  list_for_lengths = []
  for lijst in list_for_lists:
    lengte = len(lijst) + 1
    list_for_lengths.append(lengte)

  return list_for_lists, list_for_lists_values,list_for_lengths

In [22]:
def persistence_combined_new(analogue_fields,data,c_threshold):
  """
Defines the persistence of each analogue based on a correlation threshold (>=). It returns a list with a list for each analogue, -1 is a day before the event, 1 is a day after the event.
So if you want the total lenght of the event you still need to add the event itself? (nu wel in lenghts gedaan :) Also returns the correlations in the correct order.
(als je dus de analogues bepaald heb met data gesliced op seizoen die ook hier gebruiken?)
New omdat vorige met yearlenght deed, maar nu doet al semi data dus dan hoeft dat niet (en dan filtered yearlenght niet meer 90)
  """
  list_for_lists = [] #
  list_for_lists_values = []
  analogue_indexes = []

  for analogue_field in analogue_fields:
    index_counter = 0
    match_condition = False
    while match_condition == False and index_counter < (data.shape[0]):
      data_field = data[index_counter]
      if np.array_equal(data_field, analogue_field) == True:
        analogue_indexes.append(index_counter)
        match_condition = True
      else:
        index_counter = index_counter + 1
        if index_counter == (data.shape[0]):
          print ("WARNING: for persistence combined the analogue field was not found in the data (zelf)")


  # nu 500 ook laatste dag dus kan niet vooruit
  for index in analogue_indexes:
    list_for_length = []
    list_for_values = []
    analogue_map = data[index]
    Ab,Ac = np.shape(analogue_map)
    analogue_af = analogue_map.reshape(Ab*Ac)

    index_dag1 = index + 1
    index_dag2 = index - 1
    correlation = 1

    while correlation >= c_threshold and index_dag2 >= 0:
      day_data = data[index_dag2]
      Bb,Bc = np.shape(day_data)
      day_map_af = day_data.reshape(Bb*Bc)
      correlation = pearsonr(analogue_af, day_map_af)[0]
      if correlation >= c_threshold:
        list_for_length.insert(0,-1)
        list_for_values.insert(0,correlation) # dit is met insert dat is anders, dan doet ie het aan de voorkant dus dan staan de values echt op volgorde van de dag.
      index_dag2 = index_dag2 - 1

    correlation = 1

    while correlation >= c_threshold and index_dag1 <= (len(data)-1):
      day_data = data[index_dag1]
      Bb,Bc = np.shape(day_data)
      day_map_af = day_data.reshape(Bb*Bc)
      correlation = pearsonr(analogue_af, day_map_af)[0]
      if correlation >= c_threshold:
        list_for_length.append(1)
        list_for_values.append(correlation)
      index_dag1 = index_dag1 + 1


    list_for_lists.append(list_for_length) # nu dus niet anaologue zelf meegenomen in lengte dus voor lengte nog lengte lijst +1
    list_for_lists_values.append(list_for_values)

  list_for_lengths = []
  for lijst in list_for_lists:
    lengte = len(lijst) + 1
    list_for_lengths.append(lengte)

  return list_for_lists, list_for_lists_values,list_for_lengths

In [23]:
def plot_trend_combined(y_data_combined,running_mean_data_combined, running_mean_window, start_year, end_year, folder, name):
  """
Moet wel goed checken dat length x_ticks and y_data hetzelfde is (dus 100 years, is 1850 tot en met 1949! dus ook end_year = 1949)
  """
  fig, ax1 = plt.subplots(figsize = (10,8))
  x_ticks = range(start_year, end_year + 1)
  ax1.set_ylabel("1 - (ED/EDmax)")
  ax1.set_xlabel("Years")

  array_for_mean_y_data = np.array(y_data_combined)
  mean_trend_y_data = np.mean(array_for_mean_y_data, axis=0)
  
  for i in range(len(y_data_combined)):
    ax1.plot(x_ticks,y_data_combined[i], alpha = 0.3, c = "darkturquoise")
    #ax1.plot(range(start_year + (running_mean_window // 2 - 1), end_year - (running_mean_window // 2 - 1)),running_mean_data_combined[i], alpha = 0.3) #begin van bram, omdat eerste paar jaar niet window kan doen
  ax1.plot(x_ticks,mean_trend_y_data, c = "black",linewidth = 2)
  plt.savefig(f"{folder}/{name}.png",dpi=300)
  #plt.show()
  plt.close()

In [24]:
def plot_trend_combined_subplots(y_data_combined1850,y_data_combined1950, running_mean_window, start_year1850,start_year1950, end_year1850,end_year1950, folder, name):
  """
Moet wel goed checken dat length x_ticks and y_data hetzelfde is (dus 100 years, is 1850 tot en met 1949! dus ook end_year = 1949)
  """
  fig, (ax1,ax2) = plt.subplots(nrows = 1, ncols = 2,figsize = (10,4), sharey = True)
  x_ticks1 = range(start_year1850, end_year1850 + 1)
  x_ticks2 = range(start_year1950, end_year1950 + 1)
  ax1.set_ylabel("1 - (ED/EDmax)")
  ax1.set_xlabel("Years")
  ax2.set_xlabel("Years")
  
  array_for_mean_y_data1850 = np.array(y_data_combined1850)
  mean_trend_y_data1850 = np.mean(array_for_mean_y_data1850, axis=0)

  array_for_mean_y_data1950 = np.array(y_data_combined1950)
  mean_trend_y_data1950 = np.mean(array_for_mean_y_data1950, axis=0)

  for i in range(len(y_data_combined1850)):
    ax1.plot(x_ticks1,y_data_combined1850[i], alpha = 0.3, c = "darkturquoise")
    #ax1.plot(range(start_year + (running_mean_window // 2 - 1), end_year - (running_mean_window // 2 - 1)),running_mean_data_combined[i], alpha = 0.3) #begin van bram, omdat eerste paar jaar niet window kan doen
  ax1.plot(x_ticks1,mean_trend_y_data1850, c = "black",linewidth = 2)

  for i in range(len(y_data_combined1950)):
    ax2.plot(x_ticks2,y_data_combined1950[i], alpha = 0.3, c = "darkturquoise")
    #ax2.plot(range(start_year + (running_mean_window // 2 - 1), end_year - (running_mean_window // 2 - 1)),running_mean_data_combined[i], alpha = 0.3) #begin van bram, omdat eerste paar jaar niet window kan doen
  ax2.plot(x_ticks2,mean_trend_y_data1950, c = "black",linewidth = 2)

  plt.subplots_adjust(wspace=0.1) # Adjust the horizontal space between the two plot (in percentage compared to the average axis-length?)
  plt.savefig(f"{folder}/{name}.png",dpi=300)
  #plt.show()
  plt.close()

In [25]:
def violin_plot_overlay(Tevent_past,Tevent_present,Tanalogue_past,Tanalogue_present,persistence_past,persistence_present,all_t_analogue_list1850,all_t_analogue_list1950,all_lengths_list1850,all_lengths_list1950,folder,name):
  """
Doet het? maar stipjes van persistence standaard op 1 want hebt nog niet van event?
  """
  fig, (ax1,ax2) = plt.subplots(1,2,figsize = (8, 5))
  for i in range(len(all_t_analogue_list1850)):
    violins_background = ax1.violinplot([all_t_analogue_list1850[i], all_t_analogue_list1950[i]], [1, 1.6], showmeans=False, showextrema=False, showmedians=False)
    #color_background = "darkturquoise" 
    colors = ["magenta", "green"]

    #alpha_background = 0.3 
    violins_background["bodies"][0].set_facecolor(colors[0])
    #violins_background["bodies"][0].set_alpha(alpha_background)
    violins_background["bodies"][1].set_facecolor(colors[1])
    #violins_background["bodies"][1].set_alpha(alpha_background)
  
  u, p = stats.ttest_ind(Tanalogue_past, Tanalogue_present) #calculate p value
  text_to_plot = f"p = {p:.3f}"

  violins = ax1.violinplot([Tanalogue_past, Tanalogue_present], [1, 1.6], showmeans=False, showextrema=False, showmedians=False)
  colors = ["magenta", "green"]
  for pc, color in zip(violins["bodies"], colors): #pc zijn de violins dus moet zo als verschillende kleuren wil
    pc.set_facecolor(color)
    pc.set_edgecolor('black')


  ax1.plot(1,Tevent_past, marker = "o", color = "r") #plot de events
  ax1.plot(1.6,Tevent_present, marker = "o", color = "r")  #plot de events
  ax1.axhline(np.mean(Tanalogue_past), color = colors[0], linewidth = 3)
  ax1.axhline(np.mean(Tanalogue_present), color = colors[1], linewidth = 3)
  ax1.set_xticks([1, 1.6])
  ax1.set_xticklabels(["Past", "Present"])
  ax1.set_ylim(top = (1.035*(max(max(Tanalogue_past),max(Tanalogue_present))))) # to ensure that the plotted p-value and graph do not overlap
  ax1.text(0.035,0.95,text_to_plot,transform=ax1.transAxes)

  for i in range(len(all_lengths_list1850)):
    violins_background2 = ax2.violinplot([all_lengths_list1850[i], all_lengths_list1950[i]], [1, 1.6], showmeans=False, showextrema=False, showmedians=False)
    #color_background2 = "darkturquoise" 
    #alpha_background2 = 0.3 
    violins_background2["bodies"][0].set_facecolor(colors[0])
    #violins_background2["bodies"][0].set_alpha(alpha_background2)
    violins_background2["bodies"][1].set_facecolor(colors[1])
    #violins_background2["bodies"][1].set_alpha(alpha_background2)


  u, p = stats.ttest_ind(persistence_past, persistence_present) #calculate p value
  text_to_plot = f"p = {p:.3f}"

  violins = ax2.violinplot([persistence_past, persistence_present], [1, 1.6], showmeans=False, showextrema=False, showmedians=False)
  colors = ["magenta", "green"]
  for pc, color in zip(violins["bodies"], colors): #pc zijn de violins dus moet zo als verschillende kleuren wil
    pc.set_facecolor(color)
    pc.set_edgecolor('black')
    #pc.set_alpha(1)
  
  ax2.plot(1,1, marker = "o", color = "r") #plot de events
  ax2.plot(1.6,1, marker = "o", color = "r")  #plot de events
  ax2.axhline(np.mean(persistence_past), color = colors[0], linewidth = 3)
  ax2.axhline(np.mean(persistence_present), color = colors[1], linewidth = 3)
  ax2.set_xticks([1, 1.6])
  ax2.set_xticklabels(["Past", "Present"])
  ax2.set_ylim(top = (1.07*(max(max(persistence_past),max(persistence_present))))) # to ensure that the plotted p-value and graph do not overlap
  ax2.text(0.035,0.95,text_to_plot,transform=ax2.transAxes)

  plt.savefig(f"{folder}/{name}.png",dpi=300)
  plt.close()
  #plt.show()

In [26]:
def lists_for_model_dates(amount_of_years):
    """
    Creates lists with all months and years in the complete model data (for 1950-2014) (amount_of_years 65 of 100)
    """
    month_list = []
    year_list = []
    
    if amount_of_years == 65:
        year = 1950
    elif amount_of_years == 100:
        year = 1850
    else:
        print("WARNING: not a proper amount of years is given (zelf)")
    for j in range(amount_of_years):
        month = 1
        for m in range(12):
            for d in range(30):
                year_list.append(year)
                month_list.append(month)
            month = month + 1     
        year = year + 1

    return month_list, year_list

In [27]:
def extract_years_and_months_era5(era5_data,desired_start_month,desired_end_month,desired_start_year,desired_end_year,list_with_all_months,list_with_all_years):
    """
    Slice era5 data based on the months and years, months and years that are used as input variable will be included as well
    """
    if era5_data.shape[0] != len(list_with_all_months):
        print ("Error: Amount of days in the data and list with all dates are not the same")
    
    list_for_filtered_era5_data = []
    list_for_filtered_years = []
    list_for_filtered_months = []
    for i in range(era5_data.shape[0]):
        month_at_index = list_with_all_months[i]
        year_at_index = list_with_all_years[i]
        if month_at_index >= desired_start_month and month_at_index <= desired_end_month and year_at_index >= desired_start_year and year_at_index <= desired_end_year:
            data_to_select = era5_data[i,:,:]
            list_for_filtered_era5_data.append(data_to_select)
            list_for_filtered_years.append(year_at_index)
            list_for_filtered_months.append(month_at_index)
    array_selected_era5_data = np.array(list_for_filtered_era5_data)

    return array_selected_era5_data, list_for_filtered_years, list_for_filtered_months

In [28]:
def prep_streamfunction(data):
    """
    doen nadat bijgesneden
    """
    list_for_new_data = []
    for i in range(data.shape[0]):
        data_day = data[i,:,:]
        day_mean = np.mean(data_day)
        new_data = data_day - day_mean
        list_for_new_data.append(new_data)
    new_array = np.array(list_for_new_data)
    
    return new_array

In [29]:
def violin_plot2014(Tevent_past,Tevent_present,Tanalogue_past,Tanalogue_present,persistence_past,persistence_present,persistence_event,e):
  """
Doet het? maar stipjes van persistence standaard op 1 want hebt nog niet van event?
  """
  u, p = stats.ttest_ind(Tanalogue_past, Tanalogue_present) #calculate p value
  text_to_plot = f"p = {p:.3f}"

  fig, (ax1,ax2) = plt.subplots(1,2,figsize = (7, 4))
  violins = ax1.violinplot([Tanalogue_past, Tanalogue_present], [1, 1.6], showmeans=False, showextrema=False, showmedians=False)
  colors = ["magenta", "green"]
  for pc, color in zip(violins["bodies"], colors): #pc zijn de violins dus moet zo als verschillende kleuren wil
    pc.set_facecolor(color)
  ax1.axhline(np.mean(Tanalogue_past), color = colors[0], linewidth = 3)
  ax1.axhline(np.mean(Tanalogue_present), color = colors[1], linewidth = 3)
  ax1.plot(1,Tevent_past, marker = "o", color = "r") #plot de events
  ax1.plot(1.6,Tevent_present, marker = "o", color = "r")  #plot de events
  ax1.set_xticks([1, 1.6])
  ax1.set_xticklabels(["Past", "Present"])
  ax1.set_ylim(bottom=1.5e-10,top = 3.0e-10) # to ensure that the plotted p-value and graph do not overlap
  ax1.text(0.035,0.95,text_to_plot,transform=ax1.transAxes)
  ax1.set_ylabel("Typicality (x$10^{-10}$)")
  ax1.set_title("(A)",loc= "center",fontsize = 11)
  u, p = stats.ttest_ind(persistence_past, persistence_present) #calculate p value
  text_to_plot = f"p = {p:.3f}"


  violins = ax2.violinplot([persistence_past, persistence_present], [1, 1.6], showmeans=False, showextrema=False, showmedians=False)
  colors = ["magenta", "green"]
  for pc, color in zip(violins["bodies"], colors): #pc zijn de violins dus moet zo als verschillende kleuren wil
    pc.set_facecolor(color)
  ax2.axhline(np.mean(persistence_past), color = colors[0], linewidth = 3)
  ax2.axhline(np.mean(persistence_present), color = colors[1], linewidth = 3)
  ax2.plot(1,persistence_event, marker = "o", color = "r") #plot de events
  ax2.plot(1.6,persistence_event, marker = "o", color = "r")  #plot de events
  ax2.set_xticks([1, 1.6])
  ax2.set_xticklabels(["Past", "Present"])
  ax2.set_ylim(top = 11) # to ensure that the plotted p-value and graph do not overlap
  ax2.text(0.035,0.95,text_to_plot,transform=ax2.transAxes)
  ax2.set_ylabel("Persistence (days)")
  ax2.set_title("(B)",loc= "center",fontsize = 11)

  fig.suptitle(f"HadGEM3 (r{e})")
  plt.subplots_adjust(wspace=0.25)  # Change the value as needed
  #plt.savefig(f"/usr/people/noest/stage_folders/outputs/net/serious_run2/violin/violin_had{e}.png",dpi=300)
  plt.show()
  plt.close()


In [30]:
def intensity_plot(lat,lon,past,present,difference,sigmask,naam):
    fig = plt.figure(figsize = (14,3))
    gs = GridSpec(1, 6, figure=fig, height_ratios=[1], width_ratios=[1, 1,1,1,1,1])
    #ax = plt.axes(projection = ccrs.PlateCarree())

    levels1 = np.linspace(-1.75*1e7,1.75*1e7,15)
    levels2 = np.linspace(-3*1e6,3*1e6,13)
    # ax1 = fig.add_subplot(gs[0, 1:3],projection=ccrs.PlateCarree())
    # contour1 = ax1.contourf(lon, lat, event_box[0], cmap = "RdBu_r", transform = ccrs.PlateCarree(), levels = levels1,extend = "both") #levels=np.linspace(-8.2e7, 1e7, 10), extend='both
    # ax1.coastlines()
    # ax1.add_feature(cf.BORDERS)
    # ax1.set_title("A) Event",fontsize = 10)
    #plt.colorbar(ax1, ax=ax1, orientation = "horizontal", label = "Pressure (Pa)", pad = 0.05)

    ax2 = fig.add_subplot(gs[0, 0:2],projection=ccrs.PlateCarree())
    contour1 = ax2.contourf(lon, lat, past, cmap = "RdBu_r", transform = ccrs.PlateCarree(), levels = levels1,extend = "both") #levels=np.linspace(-8.2e7, 1e7, 10), extend='both
    ax2.coastlines()
    ax2.add_feature(cf.BORDERS)
    ax2.set_title("B) Past",fontsize = 10)

    ax3 = fig.add_subplot(gs[0, 2:4],projection=ccrs.PlateCarree())
    contour1 = ax3.contourf(lon, lat, present, cmap = "RdBu_r", transform = ccrs.PlateCarree(), levels = levels1,extend = "both") #levels=np.linspace(-8.2e7, 1e7, 10), extend='both
    ax3.coastlines()
    ax3.add_feature(cf.BORDERS)
    ax3.set_title("C) Present",fontsize = 10)

    ax4 = fig.add_subplot(gs[0, 4:],projection=ccrs.PlateCarree())
    contour2 = ax4.contourf(lon, lat, difference, cmap = "PiYG", transform = ccrs.PlateCarree(), levels = levels2, extend = "both") #levels=np.linspace(-8.2e7, 1e7, 10), extend='both
    ax4.contourf( lon, lat,sigmask, levels=[-0.5,0.5,1.5], hatches=[None, '////'], colors='none', transform=ccrs.PlateCarree())
    ax4.coastlines()
    ax4.add_feature(cf.BORDERS)
    ax4.set_title("D) Difference",fontsize = 10)

    title = f"HadGEM3 (r{naam})"
    fig.text(0.1, 0.5, title, va='center', ha='center', rotation='vertical', fontsize=12)


    #cax1 = fig.add_subplot(gs[2,1:3])
    #plt.colorbar(contour1, cax=cax1, orientation="horizontal", label="500 hPa Streamfunction\n(x$10^7$ m$^2$/s)")
    # formatter = ScalarFormatter(useMathText=True)
    # formatter.set_powerlimits((0, 0))  # Set the exponent limits
    # cax1.yaxis.set_major_formatter(formatter)

    #cax2 = fig.add_subplot(gs[2,4:])
    #plt.colorbar(contour2, cax=cax2, orientation="horizontal", label="Difference in 500 hPa streamfunction\n(x$10^6$ m$^2$/s)")
    #plt.savefig(f"/usr/people/noest/stage_folders/outputs/net/serious_run2/intensity/int_had{naam}.png",dpi=300)
    #plt.subplots_adjust(hspace=0.9)  # Change the value as needed
    plt.show()
    plt.close()

## **AMO**

In [31]:
def extract_amo_region_and_time_and_mean(amo_lat,amo_lon,amo_var, bbox,start_month_amo,end_month_amo):
    """
    Extract mean of a sub-set of a cube inside a lon, lat bounding box
    bbox=[lon_min lon_max lat_min lat_max].
    NOTE: This is a work around too subset an iris cube that has
    2D lon, lat coords. zelf aangepast zonder iris.
    gehardcode dat jaar 12 dingen duurt en dat alle jaren wilt
    """
    #vikki en zelf aangepast dat geen iris:
    minmax = lambda x: (np.min(x), np.max(x))
    inregion = np.logical_and(np.logical_and(amo_lon > bbox[0],
                                             amo_lon < bbox[1]),
                              np.logical_and(amo_lat > bbox[2],
                                             amo_lat < bbox[3]))
    region_inds = np.where(inregion)
    imin, imax = minmax(region_inds[0])
    jmin, jmax = minmax(region_inds[1])
    subcube_var = amo_var[..., imin:imax+1, jmin:jmax+1]
    subcube_lat = amo_lat[..., imin:imax+1, jmin:jmax+1]
    subcube_lon = amo_lon[..., imin:imax+1, jmin:jmax+1]
    
    #bram:
    years_amo = subcube_var.shape[0] // 12
    var_time_filtered = subcube_var.reshape(years_amo, 12, subcube_var.shape[1], subcube_var.shape[2])[:, start_month_amo - 1: end_month_amo, :, :]
    amo_mean_state = np.mean(var_time_filtered, axis = (1, 2, 3))
    #x = subcube.collapsed(['cell index along second dimension','cell index along first dimension'],iris.analysis.MEAN)
    #return var_time_filtered,subcube_lat,subcube_lon
    return amo_mean_state

In [32]:
def AMO_analysis(AMO_data_path_past,AMO_data_path_present,start_month_amo,end_month_amo):
    """
    Analysis the AMO data, for now it is hardcoded that you load the tos/lat/lon variables.
    bbox=[lon_min lon_max lat_min lat_max].
    nog gehardcode extract_amo_region_and_time dat jaar 12 maanden duurt en dat alle jaren wilt

    """
    amo_lat_past,amo_lon_past,amo_tos_past = load_data(AMO_data_path_past,"latitude","longitude","tos")
    amo_lat_present,amo_lon_present,amo_tos_present = load_data(AMO_data_path_present,"latitude","longitude","tos")

    NA_past_mean = extract_amo_region_and_time_and_mean(amo_lat_past,amo_lon_past,amo_tos_past, [-80, 0, 0, 60],start_month_amo,end_month_amo)
    NA_present_mean = extract_amo_region_and_time_and_mean(amo_lat_present,amo_lon_present,amo_tos_present, [-80, 0, 0, 60],start_month_amo,end_month_amo)

    global_past_mean = extract_amo_region_and_time_and_mean(amo_lat_past,amo_lon_past,amo_tos_past, [-180, 180, -60, 60],start_month_amo,end_month_amo)
    global_present_mean = extract_amo_region_and_time_and_mean(amo_lat_present,amo_lon_present,amo_tos_present, [-180, 180, -60, 60],start_month_amo,end_month_amo)

    AMO_value_past = NA_past_mean - global_past_mean #should be an array
    AMO_value_present = NA_present_mean - global_present_mean
    #AMO_value_present =  global_present_mean - NA_present_mean #test

    return AMO_value_past, AMO_value_present

In [33]:
def amo_bram_plot_test(yearlyEuclidianDistance, runningMean, amoData,running_mean_window,folder,name):
    fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, figsize = (14, 6))
    slope, intercept, r2, _, _ = scipy.stats.linregress(yearlyEuclidianDistance, amoData)
    xData = np.linspace(np.min(yearlyEuclidianDistance), np.max(yearlyEuclidianDistance), yearlyEuclidianDistance.shape[0])
    yData = [slope * x + intercept for x in xData]
    ax1.scatter(yearlyEuclidianDistance, amoData)
    ax1.plot(xData, yData)
    ax1.text(0.05, 0.95, f"r-value = {r2:.3f}", transform = ax1.transAxes, bbox = dict(facecolor = "None", edgecolor = "k"))
    ax1.set_xlabel("Euclidian distance")
    ax1.set_ylabel("AMO")
    ax1.set_title("AMO")

    running_mean_amo = np.convolve(amoData, np.ones(running_mean_window) / running_mean_window, mode = "valid")    
    slope, intercept, r2, _, _ = scipy.stats.linregress(runningMean, running_mean_amo)
    xData = np.linspace(np.min(runningMean), np.max(runningMean), runningMean.shape[0])
    yData = [slope * x + intercept for x in xData]
    ax2.scatter(runningMean, running_mean_amo)
    ax2.plot(xData, yData)
    ax2.text(0.05, 0.95, f"r-value = {r2:.3f}", transform = ax2.transAxes, bbox = dict(facecolor = "None", edgecolor = "k"))
    ax2.set_xlabel("Euclidian distance")
    ax2.set_ylabel("AMO")
    ax2.set_title("10-yr AMO")
    plt.savefig(f"{folder}/{name}.png",dpi=300)
    plt.close()


# **Combined**

In [34]:
def analogues_combined(base_path,name_run,data_path,event_path,S,N,W,E,yearlength_original,start_month,end_month,start_year1850,end_year1850,start_year1950,end_year1950,target_number_of_analogues,analogue_seperation_range,c_threshold,yearlength_filtered,running_mean_window,start_x_axis_trend1850,end_x_axis_trend1850,start_x_axis_trend1950,end_x_axis_trend1950,amount_of_analogues_to_plot,p_value_difference_maps,amo_data_folder,start_month_amo,end_month_amo,save_violin_plot = False,save_trend_plot = False, amount_of_ensembles_to_run = 55,save_analogue_plot = False,save_differences_maps = False,save_all_data = False,save_amo_plot = False):
  """
Combine all functions into a single function. For now, assumed that the number of analogues and analogue seperation range are the same for the original analogue and the analogue analogues in the typicallity.
Function is also made for the model data. If ERA5 data is used, the event itself still needs to be excluded from the analogues.
  """
  #Create paths
  new_base_path,output_folder_figures,output_folder_lists,violin_plot_path,analogue_plot_path,difference_plot_path,trend_plot_path,AMO_plot_path = create_directories(base_path,name_run,extra_folder1 = "figures",extra_folder2 = "lists")

  #Make a list to save all the sliced data for the combined violin plot (so you can look for analogues in ALL data)
  list_for_all_selected_data_1850 = []
  list_for_all_selected_data_1950 = []

  #Make lists so save the past and present analogues for the difference ensembles as well as their euclidian distances
  list_for_combined_difference_1850 = [] # Is a list of lists, with in each inner list all the analogue fields for an ensemble member
  list_for_combined_difference_1950 = []
  list_for_all_analogues_distances_1850 = []
  list_for_all_analogues_distances_1950 = []

  #Make lists for the OLD difference plot using the mean of ALL analogues (rather than the combined best ones) for all ensemble members
  list_for_old_difference_plot_1850 = []
  list_for_old_difference_plot_1950 = []

  #Make lists for the combined trend plot
  list_for_y_data_combined_trend_plot1850 = []
  list_for_running_mean_combined_trend_plot1850 = []
  list_for_y_data_combined_trend_plot1950 = []
  list_for_running_mean_combined_trend_plot1950 = []

  #Make lists for the combined violin plot
  list_for_violin_plot_t_analogues1850 = []
  list_for_violin_plot_t_analogues1950 = []
  list_for_violin_plot_lengths1850 = []
  list_for_violin_plot_lengths1950 = []

  #Make lists for combined months and years list for the new combined violin plot
  list_with_combined_years_p1 = []
  list_with_combined_months_p1 = []
  list_with_combined_years_p2 = []
  list_with_combined_months_p2 = []


  #Make lists with all months and years for both periods
  #month_list_p1,year_list_p1 = lists_for_model_dates(100)
  month_list_p2,year_list_p2 = lists_for_model_dates(65)

  #Prepare ensemble members
  ensemble_member_paths = sorted(os.listdir(data_path))
  index_1950 = 0
  max_index = amount_of_ensembles_to_run-1

  # Loop through ensemble members and determine paths
  while index_1950 <= max_index:
    file1950 = ensemble_member_paths[index_1950]
    data_path1950 = os.path.join(data_path, file1950)
    print (file1950)
    #Load data
    lat1950,lon1950,psl1950_levels = load_data(data_path1950,"lat","lon","stream")
    psl1950 = np.squeeze(psl1950_levels)
    #Load and extract area for event
    msl_event = np.load(event_path)
    lat_event_bounds,lon_event_bounds,msl_event_bounds_og = extract_area(S, N, W, E,lat1950,lon1950,msl_event,event = True)
    msl_event_bounds = prep_streamfunction(msl_event_bounds_og)

    #Extract area
    lat1950_bounds,lon1950_bounds,psl1950_bounds_og = extract_area(S, N, W, E,lat1950,lon1950,psl1950,event = False)
    psl1950_bounds = prep_streamfunction(psl1950_bounds_og)

    #Extract season nieuw #nog kijken waar season voor gebruikt wordt
    psl1850_semi, list_semi_filterd_years_p1,list_semi_filterd_months_p1 = extract_years_and_months_era5(psl1950_bounds,1,12,start_year1850,end_year1850,month_list_p2,year_list_p2)
    psl1850_season, list_season_filterd_years_p1,list_season_filterd_months_p1 = extract_years_and_months_era5(psl1950_bounds,start_month,end_month,start_year1850,end_year1850,month_list_p2,year_list_p2)

    psl1950_semi, list_semi_filterd_years_p2,list_semi_filterd_months_p2 = extract_years_and_months_era5(psl1950_bounds,1,12,start_year1950,end_year1950,month_list_p2,year_list_p2)
    psl1950_season, list_season_filterd_years_p2,list_season_filterd_months_p2 = extract_years_and_months_era5(psl1950_bounds,start_month,end_month,start_year1950,end_year1950,month_list_p2,year_list_p2)

    if save_all_data == True: #nog kijken
      list_for_all_selected_data_1850.extend(psl1850_semi)
      list_for_all_selected_data_1950.extend(psl1950_semi)
      list_with_combined_years_p1.extend(list_semi_filterd_years_p1)
      list_with_combined_months_p1.extend(list_semi_filterd_months_p1)
      list_with_combined_years_p2.extend(list_semi_filterd_years_p2)
      list_with_combined_months_p2.extend(list_semi_filterd_months_p2)

    #Calculate euclidian distance
    euclidian_distance1850 = euclidian_distance(psl1850_semi, msl_event_bounds)
    euclidian_distance1950 = euclidian_distance(psl1950_semi, msl_event_bounds)

    #Find analogues *nog kijken voor zekerheid waar index gebruikt word en dat met semi gebruikt word
    analogues_indexes1850, analogues_distances1850,analogues_years1850 = determine_analogues_era5(euclidian_distance1850, target_number_of_analogues,analogue_seperation_range,list_semi_filterd_months_p1,list_semi_filterd_years_p1,start_month,end_month,event_is_in_data=False)
    analogues_indexes1950, analogues_distances1950,analogues_years1950 = determine_analogues_era5(euclidian_distance1950, target_number_of_analogues,analogue_seperation_range,list_semi_filterd_months_p2,list_semi_filterd_years_p2,start_month,end_month,event_is_in_data=False)

    #nog kijken
    list_for_all_analogues_distances_1850.append(analogues_distances1850)
    list_for_all_analogues_distances_1950.append(analogues_distances1950)

    #Determine persistence
    persistence_days1850, persistence_correlations1850,lengths1850 = persistence(analogues_indexes1850,psl1850_semi,c_threshold)
    persistence_days1950, persistence_correlations1950,lengths1950 = persistence(analogues_indexes1950,psl1950_semi,c_threshold)

    #Calculate the mean difference per ensemble and the mean for all ensembles (goed)
    difference_per_ensemble, mean_array_1850, mean_array_1950,list_for_1850, list_for_1950 = mean_difference_per_ensemble(analogues_indexes1850,psl1850_semi,analogues_indexes1950,psl1950_semi)
    list_for_combined_difference_1850.append(list_for_1850)
    list_for_combined_difference_1950.append(list_for_1950)
    list_for_old_difference_plot_1850.append(mean_array_1850)
    list_for_old_difference_plot_1950.append(mean_array_1950)

    #Define the name to identify ensemble members in the saved output
    ensemble_number_name = (index_1950) + 1

    #Plot violin plot
    if save_violin_plot == True:

      #Calculate typicallity (nog kijken naar wat append)
      Tevent1850, Tanalogues1850 = typicality(analogues_distances1850, analogues_indexes1850,psl1850_semi,target_number_of_analogues,analogue_seperation_range,list_semi_filterd_months_p1,list_semi_filterd_years_p1,start_month,end_month)
      Tevent1950, Tanalogues1950 = typicality(analogues_distances1950, analogues_indexes1950,psl1950_semi,target_number_of_analogues,analogue_seperation_range,list_semi_filterd_months_p2,list_semi_filterd_years_p2,start_month,end_month)

      list_for_violin_plot_t_analogues1850.append(Tanalogues1850)
      list_for_violin_plot_t_analogues1950.append(Tanalogues1950)
      list_for_violin_plot_lengths1850.append(lengths1850)
      list_for_violin_plot_lengths1950.append(lengths1950)

      #Save plot
      violin_name = f"violin_plot_ensemble_number_{ensemble_number_name}"
      violin_plot(Tevent1850,Tevent1950,Tanalogues1850,Tanalogues1950,lengths1850,lengths1950,violin_plot_path,violin_name)
      violin_plot2014(Tevent1850,Tevent1950,Tanalogues1850,Tanalogues1950,lengths1850,lengths1950,1,ensemble_number_name)
    #Plot trends
    if save_trend_plot == True:

      #Calculate yearly minimum distance for trend analysis (needs seasonal euclidian distances)
      euclidian_distance1850_season = euclidian_distance(psl1850_season, msl_event_bounds)
      euclidian_distance1950_season = euclidian_distance(psl1950_season, msl_event_bounds)
      yearly_min_eucldidian_distance1850 = yearly_minimum_euclidian_distance(euclidian_distance1850_season, yearlength = yearlength_filtered)
      yearly_min_eucldidian_distance1950 = yearly_minimum_euclidian_distance(euclidian_distance1950_season, yearlength = yearlength_filtered)

      #Save plot
      trend_name_1850 = f"trend_plot_1850_ensemble_number_{ensemble_number_name}"
      trend_name_1950 = f"trend_plot_1950_ensemble_number_{ensemble_number_name}"
      y_data_to_save1850, running_mean_to_save1850 = plot_trend(yearly_min_eucldidian_distance1850,euclidian_distance1850, running_mean_window, start_x_axis_trend1850, end_x_axis_trend1850,trend_plot_path,trend_name_1850)
      y_data_to_save1950, running_mean_to_save1950 = plot_trend(yearly_min_eucldidian_distance1950,euclidian_distance1950, running_mean_window, start_x_axis_trend1950, end_x_axis_trend1950,trend_plot_path,trend_name_1950)

      list_for_y_data_combined_trend_plot1850.append(y_data_to_save1850)
      list_for_running_mean_combined_trend_plot1850.append(running_mean_to_save1850)
      list_for_y_data_combined_trend_plot1950.append(y_data_to_save1950)
      list_for_running_mean_combined_trend_plot1950.append(running_mean_to_save1950)

    if save_analogue_plot == True:
      for i in range(amount_of_analogues_to_plot):
        index_plot_1850 = analogues_indexes1850[i]
        index_plot_1950 = analogues_indexes1950[i]
        analogue_plot_name_1850 = f"analogue_plot_1850_ensemble_number_{ensemble_number_name}_analogue_rank_{i+1}" # So rank 1 is the best analogue
        analogue_plot_name_1950 = f"analogue_plot_1950_ensemble_number_{ensemble_number_name}_analogue_rank_{i+1}" # So rank 1 is the best analogue
        plot_variable(lat1950_bounds,lon1950_bounds,psl1850_semi[index_plot_1850],analogue_plot_path,analogue_plot_name_1850)
        plot_variable(lat1950_bounds,lon1950_bounds,psl1950_semi[index_plot_1950],analogue_plot_path,analogue_plot_name_1950)

    if save_differences_maps == True:
      significance_mask = t_test(list_for_1850,list_for_1950,p_value_difference_maps)
      difference_map_name = f"difference_plot_ensemble_number_{ensemble_number_name}"
      plot_difference(lat1950_bounds,lon1950_bounds,difference_per_ensemble,significance_mask,difference_plot_path,difference_map_name)
      intensity_plot(lat1950_bounds,lon1950_bounds,mean_array_1850,mean_array_1950,difference_per_ensemble,significance_mask,ensemble_number_name)




    if save_amo_plot == True:
      amo_data_paths = sorted(os.listdir(amo_data_folder))
      amo_file1850 = amo_data_paths[index_1950]
      amo_file1950 = amo_data_paths[index_1950]
      amo_data_path1850 = os.path.join(amo_data_folder, amo_file1850)
      amo_data_path1950 = os.path.join(amo_data_folder, amo_file1950)
      amo_values_1850, amo_values_1950 = amo_data_test1, amo_data_test2= AMO_analysis(amo_data_path1850,amo_data_path1950,start_month_amo,end_month_amo)
      amo_1850_name = f"amo_plot_1850_ensemble_number_{ensemble_number_name}"
      amo_1950_name = f"amo_plot_1950_ensemble_number_{ensemble_number_name}"

      amo_bram_plot_test(y_data_to_save1850, running_mean_to_save1850, amo_values_1850,running_mean_window,AMO_plot_path,amo_1850_name)
      amo_bram_plot_test(y_data_to_save1950, running_mean_to_save1950, amo_values_1950,running_mean_window,AMO_plot_path,amo_1950_name)

    
    #Increase index:
    index_1950 = index_1950 + 1

  #Outside of loop again:
  


  #This is plotting the difference of all ensembles combined by using the mean from all analogues from all ensemble members and avereging those again, does not contain significance yet so therefore also uses plot_variable instead of plot_difference
  array_means_1850 = np.array(list_for_old_difference_plot_1850) #maak van de list weer een array
  ensembles_combined_mean_array_1850_old = np.mean(array_means_1850, axis = 0)
  array_means_1950 = np.array(list_for_old_difference_plot_1950) #maak van de list weer een array
  ensembles_combined_mean_array_1950_old = np.mean(array_means_1950, axis = 0)
  combined_difference_old = ensembles_combined_mean_array_1950_old - ensembles_combined_mean_array_1850_old
  if save_differences_maps == True:
    combined_difference_map_name_old = "difference_plot_all_ensembles_combined_old"
    plot_variable(lat1950_bounds,lon1950_bounds,combined_difference_old,difference_plot_path,combined_difference_map_name_old)

  #This plots the difference maps using only the TOP analogues of all ensemble members combined (goed)
  ensembles_combined_mean_array1850_new, best_combined_analogue_fields1850, best_combined_distances_list1850 = combined_difference_top_analogues(list_for_combined_difference_1850,list_for_all_analogues_distances_1850,target_number_of_analogues)
  ensembles_combined_mean_array1950_new, best_combined_analogue_fields1950, best_combined_distances_list1950 = combined_difference_top_analogues(list_for_combined_difference_1950,list_for_all_analogues_distances_1950,target_number_of_analogues)
  combined_difference_new = ensembles_combined_mean_array1950_new - ensembles_combined_mean_array1850_new
  significance_mask_combined = t_test(best_combined_analogue_fields1850,best_combined_analogue_fields1950,p_value_difference_maps)

  if save_differences_maps == True:
    combined_difference_map_name_new = "difference_plot_all_ensembles_combined_new"
    plot_difference(lat1950_bounds,lon1950_bounds,combined_difference_new,significance_mask_combined,difference_plot_path,combined_difference_map_name_new)
  
  #Create the combined violin plot and the necessary combined data
  if save_all_data == True:
    all_selected_data_1850_array = np.array(list_for_all_selected_data_1850)
    all_selected_data_1950_array = np.array(list_for_all_selected_data_1950)
    Tevent1850_combined, Tanalogues1850_combined = typicality_combined_new(best_combined_distances_list1850,best_combined_analogue_fields1850,all_selected_data_1850_array,target_number_of_analogues,analogue_seperation_range,list_with_combined_months_p1,list_with_combined_years_p1,start_month,end_month)
    Tevent1950_combined, Tanalogues1950_combined = typicality_combined_new(best_combined_distances_list1950,best_combined_analogue_fields1950,all_selected_data_1950_array,target_number_of_analogues,analogue_seperation_range,list_with_combined_months_p2,list_with_combined_years_p2,start_month,end_month)
    persistence_days1850_combined, persistence_correlations1850_combined,lengths1850_combined = persistence_combined_new(best_combined_analogue_fields1850,all_selected_data_1850_array,c_threshold)
    persistence_days1950_combined, persistence_correlations1950_combined,lengths1950_combined = persistence_combined_new(best_combined_analogue_fields1950,all_selected_data_1950_array,c_threshold)
    violin_name_combined = f"violin_plot_combined"
    violin_name_combined_overlay = f"violin_plot_combined_overlay"
    violin_plot(Tevent1850_combined,Tevent1950_combined,Tanalogues1850_combined,Tanalogues1950_combined,lengths1850_combined,lengths1950_combined,violin_plot_path,violin_name_combined)
    violin_plot_overlay(Tevent1850_combined,Tevent1950_combined,Tanalogues1850_combined,Tanalogues1950_combined,lengths1850_combined,lengths1950_combined,list_for_violin_plot_t_analogues1850,list_for_violin_plot_t_analogues1950,list_for_violin_plot_lengths1850,list_for_violin_plot_lengths1950,violin_plot_path,violin_name_combined_overlay)

  #Create the combined trend plot
  if save_trend_plot == True:
    trend_name_1850_combined = f"combined_trend_plot_1850"
    trend_name_1950_combined = f"combined_trend_plot_1950"
    trend_name_combined_subplots = f"combined_trend_plot_subplots"

    plot_trend_combined(list_for_y_data_combined_trend_plot1850,list_for_running_mean_combined_trend_plot1850, running_mean_window, start_x_axis_trend1850, end_x_axis_trend1850,trend_plot_path,trend_name_1850_combined)
    plot_trend_combined(list_for_y_data_combined_trend_plot1950,list_for_running_mean_combined_trend_plot1950, running_mean_window, start_x_axis_trend1950, end_x_axis_trend1950,trend_plot_path,trend_name_1950_combined)
    plot_trend_combined_subplots(list_for_y_data_combined_trend_plot1850,list_for_y_data_combined_trend_plot1950, running_mean_window, start_x_axis_trend1850,start_x_axis_trend1950, end_x_axis_trend1850,end_x_axis_trend1950, trend_plot_path, trend_name_combined_subplots)



# **Uitvoeren**

In [35]:
base_path = "/usr/people/noest/stage_folders/outputs"

In [36]:
# Define the name of the map where all the output will be saved
name_run = "2014_violin_test"

In [37]:
# Define the path where the data is located (path to folder)
data_path = "/net/pc200246/nobackup/users/noest/streamfunction_hadgem/regridded_merged"

In [38]:
# Define the path where the event is located (path to actual file)
event_path = "/usr/people/noest/stage_folders/event_data/Vautard_southerlyflow_2019-06-29_regridded_streamfunction_data_at_index_25381.npy"

In [39]:
# Define area of interest
S, N, W, E = 30,60,-30,20

In [40]:
# Define period (season and years (yearsfor past and present))
yearlength_original = 360 # the year length in the original data
start_monthV = 3
end_monthV = 5
start_year1850 = 1950 # als alle jaren moet False zijn = niet meer, gwn jaar invullen
end_year1850 = 1979 # als alle jaren moet False zijn
start_year1950 = 1985 # als alle jaren moet False zijn
end_year1950 = 2014 # als alle jaren moet False zijn

In [41]:
# How many analogues to find and how many days they have to be seperated by (5 is not 5 days before AND after)
target_number_of_analogues = 30
analogue_seperation_range = 6

In [42]:
# Determine the minimum correlation coefficient it needs to be taken into account as the same event
c_threshold = 0.9

In [43]:
# Yearly minimum distance for trend analysis
yearlength_filtered = 90 # For the cut down data, so if 3 months actually used for analogues = 90

In [44]:
# Save violin plot?:
save_violin_plot = False

In [45]:
# Save trend plot? (als False toch nog iets invullen voor de andere!)
save_trend_plot = False
running_mean_window = 10
start_x_axis_trend1850 = 1850 # voor alles is 1850
end_x_axis_trend1850 = 1949 # voor alles is 1949
start_x_axis_trend1950 = 1950 # voor alles is 1950
end_x_axis_trend1950 = 2014 # voor alles is 2014

In [46]:
# Save a plot of the best analogues for each ensemble?
save_analogue_plot = False
amount_of_analogues_to_plot = 30 # It will save the best x analogues as seperate plots (for 1850 and 1950)

In [47]:
# Save the differences map for all analogues combined and each analogue seperate
save_differences_maps = False
p_value_difference_maps = 0.05 # the p value for the significance in the difference plots

In [48]:
# How many ensemble members to run (#5 is everything)
amount_of_ensembles_to_run = 1

In [49]:
# Save all the selected days for the selected region to look for analogues in all data combined for the combined violin plot
save_all_data = False

In [50]:
# Determine whether to load the AMO data and save the plots
save_amo_plot = False
amo_data_folder = '/net/pc200023/nobackup/users/thompson/LESFMIP/HadGEM/hist/tos'
start_month_amo = 9
end_month_amo = 11
#start year and end year nog doen 

In [51]:
# Run the function
analogues_combined(base_path,name_run,data_path,event_path,S,N,W,E,yearlength_original,start_monthV,end_monthV,start_year1850,end_year1850,start_year1950,end_year1950,target_number_of_analogues,analogue_seperation_range,c_threshold,yearlength_filtered,running_mean_window,start_x_axis_trend1850,end_x_axis_trend1850,start_x_axis_trend1950,end_x_axis_trend1950,amount_of_analogues_to_plot,p_value_difference_maps,amo_data_folder,start_month_amo,end_month_amo,save_violin_plot = save_violin_plot,save_trend_plot = save_trend_plot,amount_of_ensembles_to_run = amount_of_ensembles_to_run,save_analogue_plot = save_analogue_plot,save_differences_maps = save_differences_maps,save_all_data=save_all_data,save_amo_plot=save_amo_plot)

psi500_day_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_1950_2014.nc


# **Proberen**